# Исследование объявлений о продаже квартир


**КРАТКОЕ ОПИСАНИЕ ИССЛЕДОВАНИЯ** 


**Задача исследования** 

Научиться определять рыночную стоимость объектов недвижимости и установить параметры, которые на нее влияют. 
Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность.

**Входные данные** 

Данные сервиса Яндекс.Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктах за несколько лет.
По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта, ближайшего парка и водоёма.

* airports_nearest — расстояние до ближайшего аэропорта в метрах (м)

* balcony — число балконов

* ceiling_height — высота потолков (м)

* cityCenters_nearest — расстояние до центра города (м)

* days_exposition — сколько дней было размещено объявление (от публикации до снятия)

* first_day_exposition — дата публикации

* floor — этаж

* floors_total — всего этажей в доме

* is_apartment — апартаменты (булев тип)

* kitchen_area — площадь кухни в квадратных метрах (м²)

* last_price — цена на момент снятия с публикации

* living_area — жилая площадь в квадратных метрах (м²)

* locality_name — название населённого пункта

* open_plan — свободная планировка (булев тип)

* parks_around3000 — число парков в радиусе 3 км

* parks_nearest — расстояние до ближайшего парка (м)

* ponds_around3000 — число водоёмов в радиусе 3 км

* ponds_nearest — расстояние до ближайшего водоёма (м)

* rooms — количество комнат

* studio — квартира-студия (булев тип)

* total_area — площадь квартиры в квадратных метрах (м²)

* total_images — число фотографий квартиры в объявлении


**Практическое применение результатов исследования**

Исследование позволит построить автоматизированную систему для отслеживания аномалий и мошеннической деятельности.

# <a name="contents"> Содержание исследования</a>

## [Обзор данных](#stage_1)
   
* импортируем библиотеки
* посмотрим на начало и конец датасета
* получим общую информацию о данных в датасете
* посмотрим, есть ли явные дубликаты в данных
* сделаем вывод

## [Предобработка данных](#stage_2)

* изменим названия некоторых столбцов 
* изменим порядок следования столбцов
* определим и изучим пропуски
* приведем данные к нужным типам
* почистим текстовые данные

## [Обогащение данных](#stage_3)

* подсчитаем стоимость 1 кв.м.
* добавим дня недели, месяца, года публикации объявления
* категоризуем данных об этаже, на котором находится квартира
* вычислим отношение жилой площади к общей
* вычислим отношение площади кухни к общей площади

## [Исследовательский анализ данных](#stage_4)

* изучим распределение значений в данных 
* определим аномалии, редкие значения
* определим, какие факторы больше всего влияют на стоимость квартиры
* найдем десять населенных пунктов с наибольшим количеством объявлений

## [Исследование данных о квартирах Санкт-Петербурга](#stage_5)
* определим, какая область входит в центр Санкт-Петербурга
* проанализируем и сравним некоторые параметры для центра Санкт-Петербурга и остальных районов города
* проанализируем факторы, влияющие на стоимость квартиры в центре Санкт-Петербурга и остальных районов города 

## [Общий вывод](#stage_6)

<a name="stage_1"></a>

# Этап 1. Обзор данных


[Наверх к оглавлению](#contents)

Импортируем библиотеки для проекта.

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

Прочитаем данные и сохраним в переменную df.

In [ ]:
df = pd.read_csv('real_estate_data.csv', sep='\t') 

Посмотрим на данные: ссылка на переменную покажет и голову, и хвост датафрейма.

In [ ]:
df

Получим общую информацию о данных в датасете.

In [ ]:
df.info()

Видны пропуски в данных. Посмотрим на описательную статистику датафрейма. Для удобства уберем числа в экспоненциальной форме. 

In [ ]:
pd.options.display.float_format = '{:.3f}'.format
df.describe().T

Посмотрим, есть ли явные дубликаты.

In [ ]:
f'Количество явных дубликатов в данных: {df.duplicated().sum()}'

Рассмотрим уникальные значения данных для остальных столбцов.

In [ ]:
for column_name in df.columns:
    if column_name not in df.describe().columns:
        print(df[column_name].value_counts(), '\n') 

## Вывод

    
* Общее количество наблюдений в данных 23699. Объекты недвижимости описываются 22 признаками (в таблице 22 столбца). 
В 14 столбцах есть пропущенные значения.
    
* В датафрейме представлены данные разных типов: булев тип (2 столбца), строка (3 столбца), дробное число (14 столбцов), целое число (3 столбца). В некоторых случаях потребуется замена типа данных, к примеру дата публикации объявления - строка, а должна быть datetime, столбец is_apartment должен быть булевым, а не строкой. 

* Названия столбцов нужно привести к одной логике, к одну регистру и сделать более информативными. 
  
* Предварительно можно утверждать, что данных достаточно для проведения исследования.   

<a name="stage_2"></a>

# Этап 2. Предобработка данных


[Наверх к оглавлению](#contents)

## Изменим названия некоторых столбцов.

Чтобы они были более информативны.

In [ ]:
df.columns

Переименуем столбцы с помощью функции rename() и словаря. 

In [ ]:
(df.rename(columns={"total_area": "total_area_m2", "rooms": "number_of_rooms", "ceiling_height":"ceiling_height_m", 
                    "floors_total": "floors_in_house", "living_area": "living_area_m2",
                   "studio": "is_studio", "open_plan": "is_open_plan", 
                    "kitchen_area": "kitchen_area_m2", "balcony": "number_of_balconies",
                    "airports_nearest": "airports_nearest_m", "cityCenters_nearest": "city_center_distance_m",
                    "parks_nearest": "parks_nearest_m", "parks_around3000": "parks_within_3000m", 
                    "ponds_nearest":"ponds_nearest_m", "ponds_around3000": "ponds_within_3000m"}, inplace=True))

# Проверяем, что получилось:
df.columns

## Изменим порядок следования столбцов.

Для удобства работы с датасетом. Логика будет такая: где, что, сколько стоит, когда и остальная информация о близости к центру, о парках, водоемах ит.д. 

In [ ]:
df.head()

In [ ]:
df = df[[
 'locality_name',  
 'total_area_m2',
 'living_area_m2',
 'kitchen_area_m2',
 'ceiling_height_m',
 'number_of_balconies',
 'number_of_rooms',
 'floor',
 'floors_in_house',
 'is_apartment',
 'is_studio',
 'is_open_plan',  
 'total_images',
 'last_price',
 'first_day_exposition',
 'days_exposition',
 'city_center_distance_m',
 'airports_nearest_m',
 'parks_nearest_m', 
 'parks_within_3000m',
 'ponds_nearest_m',
 'ponds_within_3000m'
]]

Проверяем, что получилось:

In [ ]:
df.head()

##  Определим и изучим пропущенные значения в датасете.

Посмотрим на процент пропущенных значений в нашем датафрейме и выведем в отсортированном по убыванию виде:

In [ ]:
df.isnull().mean().round(3).mul(100).sort_values(ascending=False)

<a id='corr'></a> 
Прежде чем заполнять пропуски, посмотрим на матрицу корреляции с сырыми данными c помощью метода `corr()`. 

Это поможет нам примерно понять, от чего зависит тот или иной параметр и как лучше заполнять пропуски. 

In [ ]:
#Создадим список интересных нам столбцов и по ним построим матрицу корреляции
cols = ['last_price', 'living_area_m2', 'floor', 'city_center_distance_m', 'number_of_rooms', 'ceiling_height_m', 'floors_in_house', 'days_exposition']
df[cols].corr()

Виден высокий коэффициент коррреляции между количеством комнат и жилой площадью. И это отвечает здравому смыслу. Используем это при заполнении пропусков в столбце с жилой площадью.

Для начала создадим функцию, которая позволит нам быстро проверять результаты заполнения пропусков в столбце.

In [ ]:
"""
Обе функции берут на вход строковые значения - название датафрейма и название столбца   и 
выводят на экран сообщение о количестве пропущенных значений в данном столбце данного датафрейма.
"""

def track_nan_filling_before(dataframe, column_name):
    return print(f'Количество пропущенных значений в столбце {column_name} до обработки: ', len(dataframe[dataframe[column_name].isnull()]))
    

In [ ]:
def track_nan_filling_after(dataframe, column_name):
    return print(f'Количество пропущенных значений в столбце {column_name} после обработки: ', len(dataframe[dataframe[column_name].isnull()]))

Протестируем на функцию на отдельном значении.

In [ ]:
track_nan_filling_before(df, 'living_area_m2')

### Столбец `is_apartment`

Скорее всего, пропуски в данном столбце связаны с тем, что если объект не является апартаментами, то человек ничего не проставлял в этом пункте. Заполним пропуски в таком случае булевым значением False.  

In [ ]:
track_nan_filling_before(df, 'is_apartment')
df['is_apartment'] = df['is_apartment'].fillna(False)
track_nan_filling_after(df, 'is_apartment')

Теперь значения в этом столбце имеют булев тип данных, как это и требуется. 

In [ ]:
df['is_apartment'].dtypes

### Столбец `living_area_m2`

В столбце с жилой площадью отсутствует 8% значений. 

Заполним медианой в зависимости от количества комнат. Это решение подкреплено довольно высоким [коэффициентом корреляции Пирсона](#corr), равным 0.845977.

In [ ]:
track_nan_filling_before(df, 'living_area_m2')

df['living_area_m2']= df['living_area_m2'].fillna(df.groupby(['number_of_rooms'])['living_area_m2'].transform('median'))

track_nan_filling_after(df, 'living_area_m2')

### Столбец `number_of_balconies`

Пропуски в столбце с числом балконов, вероятнее всего, обусловлены тем, что балкона в таких случаях нет. Заполним такие пропуски значением 0.

In [ ]:
track_nan_filling_before(df, 'number_of_balconies')
df['number_of_balconies'] = df['number_of_balconies'].fillna(0)
track_nan_filling_after (df, 'number_of_balconies')

### Столбцы `floors_in_house`  и `locality_name`

In [ ]:
df.columns

Пропуски в столбцах с этажностью дома и названием населенного пункта составляют очень маленький процент (0.4% и 0.2% соответственно), заполнить их более и менее точными значениями не представляется возможным. По этой причине строки с пропущенными значениями в `floors_in_house` удалим, а пропуски в `locality_name` иожно заполнить строкой 'unknown'.

In [ ]:
track_nan_filling_before(df, 'floors_in_house')
track_nan_filling_before(df, 'locality_name')

#Подаем в метод dropna список столбцов, в которых ищем NaN и удаляем строки
df.dropna(subset=['floors_in_house'], inplace=True)
df['locality_name'] = df['locality_name'].fillna('unknown')

track_nan_filling_after(df, 'floors_in_house')
track_nan_filling_after(df, 'locality_name')

### Столбец `ceiling_height_m`

Высота потолков может существенно влиять на цену недвижимости, а также пропусков в этих данных достаточно много - 38 % (если просто удалить эти строки, мы потеряем немало данных), поэтому пропуски в  столбце ceiling_height мы заполним значением медианы в зависимости от жилой площади. Как правило, чем больше площадь комнаты - тем выше в среднем потолок. И это подтверждает и коэффициент корреляции в нашей матрице корреляций: он здесь выше, чем в других случаях. Попробуем заполнить пропуски медианой значений ceiling_height в зависимости от жилой площади. 

In [ ]:
track_nan_filling_before(df, 'ceiling_height_m')

df['ceiling_height_m']= df['ceiling_height_m'].fillna(df.groupby(['living_area_m2'])['ceiling_height_m'].transform('median'))

track_nan_filling_after(df, 'ceiling_height_m')

У нас остались  пропущенные значения, потому что для каких-то значений в группировочном столбце c living_area_m2 вообще нет заполненных значений по целевому столбцу ceiling_height_m. 
Заполним в таком случае пропущенные значения общей медианой для высота потолков.

In [ ]:
#Посмотрим, чему равна медиана в столбце ceiling_height_m:
df['ceiling_height_m'].median()

In [ ]:
track_nan_filling_before(df, 'ceiling_height_m')

df['ceiling_height_m']= df['ceiling_height_m'].fillna(df['ceiling_height_m'].median())

track_nan_filling_after(df, 'ceiling_height_m')

Проверим теперь описательную статистику для высоты потолков.

In [ ]:
df["ceiling_height_m"].describe()

Значения среднего, медианы и квартилей в пределах нормы: для стандартного жилья минимальная высота потолка 2,4 м. В новостройках обычно это значение находится в пределах 2,5-2,7 метра. Для элитного жилья показатель может превышать 3 метр по данным [статьи](https://stone-floor.ru/stati/optimalnaya-visota-potolka#:~:text=%D0%9A%D0%B0%D0%BA%D0%BE%D0%B2%D0%B0%20%D1%81%D0%B8%D1%82%D1%83%D0%B0%D1%86%D0%B8%D1%8F%20%D0%B2%20%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8%3F,%D0%BF%D0%BE%D0%BA%D0%B0%D0%B7%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%20%D0%BC%D0%BE%D0%B6%D0%B5%D1%82%20%D0%BF%D1%80%D0%B5%D0%B2%D1%8B%D1%88%D0%B0%D1%82%D1%8C%203%20%D0%BC%D0%B5%D1%82%D1%80%D0%B0.).

### Столбец `days_exposition` (длительность размещения объявления)

В столбце days_exposition пропущено 13,4%. 

Посмотрим на количество пустых значений days_exposition по годам, чтобы понять, связаны ли пропуски в этом столбце с тем, что на момент импорта данных объявления были еще актуальны.

In [ ]:
#Создам копию исходного датафрейма
df_cp = df.copy() 

#Поменяю тип данных для  first_day_exposition, чтобы потом извлечь легко оттуда год
df_cp['first_day_exposition'] = pd.to_datetime(df_cp['first_day_exposition'], format='%Y-%m-%dT%H:%M:%S')

#Создаю столбец с годом 
df_cp['year'] = df_cp['first_day_exposition'].dt.year

#Группирую данные с пропусками в days_exposition по годам: 
df_cp[df_cp['days_exposition'].isnull()].groupby('year').count()


In [ ]:
print('В датасете данные за период: ', df_cp['year'].min(), '-', df_cp['year'].max())

Наибольшее число пропущенных значений для последнего года, 2019. 

Да, похоже, действительно у нас пропуски в days_exposition в основном связаны с тем, что объявления на момент выгрузки данных были актуальны. Оставляем пропущенные значения в данном столбце.

### Столбец `kitchen_area_m2`

Пропуски в столбце с площадью кухни составляют 9.6%. Эти данные необходимы для исследования и мы можем примерно логически вычислить эти значения.
Сначала заполним пропущенные значения 0, потом вычислим медиану для площади кухни и общей площади; узнаем, какую долю от общей площади составляет площадь кухни и найдем площадь кухни, умножая общую площадь на этот коэффициент. Для квартир-студий площадь кухни таким образом не рассчитываем. 

In [ ]:
track_nan_filling_before(df, 'kitchen_area_m2')
df['kitchen_area_m2'] = df['kitchen_area_m2'].fillna(0)
print ('Количество значений 0 в столбце kitchen_area_m2 после обработки: ', len(df.query('kitchen_area_m2 == 0')))

Применим метод **where** из библиотеки numpy для вычисления площади кухни.


In [ ]:
df.columns

In [ ]:
df['kitchen_area_m2'] = np.where((df['kitchen_area_m2'] == 0) & 
                               (df['is_studio'] == False), # условие исключает расчет площади кухни для квартир-студий, что логично (кухня там совмещена с комнатой)
                               df['total_area_m2'] *  (df['kitchen_area_m2'].median() / df['total_area_m2'].median()), #каким значением заменяем 
                               df['kitchen_area_m2'] # в каком столбце делаем замену
                             )

In [ ]:
print('Количество пропущенных значений в столбце kitchen_area после обработки: ', len(df[df["kitchen_area_m2"].isnull()]))

### Другие столбцы

Мы не можем подставить логическую замену или как-то вычислить пропущенные значения в столбцах ниже, данные по которым собирались автоматически: 

* parks_within_3000m 
* ponds_within_3000m 
* airports_nearest_m
* ponds_within_3000m
* parks_around3000  
* city_center_distance_m 

Мы не можем заполнить пропуски в этих случаях без искажения информации - оставим пропуски в данных столбцах.

Также посмотрим на процент пропущенных значений в столбце city_center_distance_m для Санкт-Петербурга.

In [ ]:
df[df['locality_name'] == 'Санкт-Петербург']['city_center_distance_m'].isna().mean()*100

Из всех квартир Санкт-Петербурга информация по расстоянию до центра отсутствует всего лишь для 0,4% квартир.

Проверим теперь, сколько пропущенных значений осталось в наших данных 

In [ ]:
df.isna().mean()

### Вывод


* Мы избавились от пропусков в ключевых для нашего исследования столбцах, предложили логичную замену там, где это возможно:
     

    В столбце с информацией о том, является ли объект апартаментами, заполнили нулями. 

    В столбце с жилой площадью заполнили медианой в зависимости от количества комнат. 

    В столбце с числом балконов заполнили нулями. 

    В столбце с этажностью дома пропуски составляли лишь 4% -  удалили эти строки. 

    В столбце с названием населенного пункта - заполнили строковым значением 'unknown'.

    В столбце с площадью кухни -  заполнили отношением медианы площади кухни к медиане общей площади, умноженным на общую площадь.  
    
* Пропуски в столбцах с расстоянием до ближайшего аэропорта, парка, пруда, число парков и прудов в радиусе 3 км мы решили оставить, поскольку не можем подставить логически или вычислить пропущенные значения в них без искажения реальной картины. 
  
* Пропущенные значения в столбце с расстоянием до центра мы оставляем как есть, потому что обработать корректно, не добавляя шума в данных, нельзя, при этом из всех значений в этом столбце нас будут интересовать только значения для квартир Санкт-Петербурга, а в таком случае пропущено всего лишь 0.4 %, что допустимо для данного исследования. 
   

## Приведем данные к нужным типам

Посмотрим в начале на типы данных в нашем датафрейме с помощью атрибута **dtypes**.

In [ ]:
df.dtypes

В столбце `first_day_exposition` у нас хранится дата публикации объявления, однако тип данных в столбце - object (строка). Преобразуем строку в datetime. 

Чтобы увидеть формат, посмотрим на любое значение.

In [ ]:
df['first_day_exposition'][0]

Применим метод **to_datetime** к столбцу с датой и зададим формат, к которому нужно преобразовать строку с датой.

In [ ]:
df['first_day_exposition'] = pd.to_datetime(df['first_day_exposition'], format='%Y-%m-%dT%H:%M:%S')

In [ ]:
df['first_day_exposition']

Теперь все хорошо: тип данных для даты - datetime64. 

Дальше с датой мы можем проводить разные операции: выделять день недели, месяц и год публикации. Если бы дата была строкой, то это было бы проблематично.

В столбцах `number_of_balconies`, `floors_in_house`, `days_exposition` (длительность размещения объявления) данные хранятся в виде дробных чисел, заменим на целые числа - тип int64. 

Применим метод **astype** к датафрейму и подадим в него словарь с нужными столбцами и типами данных для замены. 
Для исключения в случае пропусков выставим errors='ignore'.

In [ ]:
df = df.astype({'floors_in_house': 'int64', 'number_of_balconies': 'int64', 'days_exposition': 'int64'}, errors='ignore')

Проверим результаты преобразований типов данных.

In [ ]:
df.dtypes

## Работа с текстом (названием населенного пункта)

Прежде посмотрим количество уникальных населенных пунктов в столбце "locality_name".

In [ ]:
f"Количество уникальных населенных пунктов в нашем датасете: {len(df['locality_name'].unique())}"

In [ ]:
df['locality_name']

У нас есть два варианта написания слова "поселок" - через 'е' и 'ё'. Мы унифицируем запись: заменим везде на 'е'.

In [ ]:
df['locality_name'] = df.locality_name.str.replace('ё', 'е')

In [ ]:
df['locality_name']

In [ ]:
f"Количество уникальных населенных пунктов в нашем датасете: {len(df['locality_name'].unique())}"

Были неявные дубли в значениях столбца с населенным пунктом - исправили. 

Посмотрим, как теперь выглядит наш датафрейм после предобработки:

In [ ]:
df.info()

<a name="stage_3"></a>

# Этап 3. Обогащение данных


[Наверх к оглавлению](#contents)

## Подсчет стоимость 1 квадратного метра

Вычислим цену за 1 кв м, поделив стоимость квартиры на общую площадь и округлив стоимость с помощью функции round.

In [ ]:
df['price_per_m2'] = round(df['last_price'] / df['total_area_m2'])
df['price_per_m2']

## Добавление дня недели, месяца и года публикации объявления

Найдем день недели и запишем в отдельный столбец нашего датафрейма, используя методы **weekday, month, year** класса **datetime** :

In [ ]:
# Запишем в отдельный столбец нашего датафрейма день недели, месяц, год:
df['weekday'] = df['first_day_exposition'].dt.weekday
df['month'] = df['first_day_exposition'].dt.month
df['year'] = df['first_day_exposition'].dt.year

Проверим, что у нас получилось:

In [ ]:
df[['first_day_exposition','weekday', 'month', 'year']]

День 0 - это понедельник.

## Категоризация данных об этаже квартиры

Посмотрим в начале, какие у нас значения в столбце `floor`.

In [ ]:
df['floor'].value_counts(ascending=False).to_frame()

Категоризуем эти данные с помощью функции.

In [ ]:
"""
Функция возвращает категорию этажа по значению этажа (floor) и этажности дома (floors_in_house), используя правила:
- 'первый', если этаж 1
- 'последний',  если этаж квартиры совпадает с этажностью дома
- 'другой' во всех остальных случаях
"""

def get_floor_category(row):
    
    floor = row['floor']
    total_floors = row['floors_in_house']
    
    if floor == 1:
        return 'первый этаж'
    
    elif floor == total_floors:
        return 'последний этаж'
    
    else:
        return 'другой'

Протестируем функцию на игрушечном датафрейме.

In [ ]:
row = pd.Series(data=[9,9], index=['floor', 'floors_in_house']) 
get_floor_category(row)

Применим функцию по категоризации этажей к столбцам нашего датафрейма и запишем результат в новый столбец `floor_category`.

In [ ]:
df['floor_category'] = df.apply(get_floor_category, axis=1)

Проверим, что у нас получилось.

In [ ]:
df[['floor', 'floors_in_house', 'floor_category']]

## Отношение жилой и общей площади

In [ ]:
df.columns

Вычислим отношение жилой площади к общей. Результат запишем в новый столбец `living_to_total_area_m2`.

In [ ]:
df['living_to_total_area_m2'] = df['living_area_m2'] / df['total_area_m2']

In [ ]:
df['living_to_total_area_m2']

## Отношение площади кухни к общей площади

Вычислим отношение площади кухни к общей площади.

In [ ]:
df['kitchen_to_total_area_m2'] = df['kitchen_area_m2'] / df['total_area_m2']
df['kitchen_to_total_area_m2']

Посмотрим на обогащенный датафрейм.

In [ ]:
df.info()

Необходимые расчеты добавлены в таблицу, можно приступать к исследовательскому анализу данных.

<a name="stage_4"></a>

# Этап 4. Исследовательский анализ данных


[Наверх к оглавлению](#contents)

## Распределение значений в данных

### Стоимость квартиры

Построим с ходу гистограмму для цены с помощью библиотеки seaborn.

In [ ]:
sns.histplot(data=df, x='last_price', kde=True);

Узкий пик до 40 000 000 и дальше длинный хвост из маленького количества значений, который скрадывает все остальные данные. У нас скошенное распределение. Попробуем отсечь выбросы и посмотреть поближе.

Построим также ящик с усами, или диаграмму размаха, чтобы оценить распределение и увидеть характерный разброс

In [ ]:
df['last_price'].plot.box();

Наш ящик оказался сжат выбросами, которых много и все они лежат выше верхнего межквартильного размаха. Уберем выбросы. 
Посмотрим, с какими значениями будем работать c помощью метода describe:

In [ ]:
df['last_price'].describe()

Посчитаем первый квартиль, третий квартиль и вычислим межквартильный размах.

In [ ]:
q1 = df['last_price'].quantile(0.25)
q3 = df['last_price'].quantile(0.75)
iqr = q3 - q1

In [ ]:
f'Первый квартиль, 25%: {q1}'

In [ ]:
f'Третий квартиль, 75%: {q3}'

In [ ]:
f'Межквартильный размах, расстояние между третьим квартилем и первым: {iqr}'

In [ ]:
f"Не более 1% квартир имеют стоимость жилья выше {np.percentile(df['last_price'], 99)} рублей"

Сделаем срез: уберем из нашего датасета все данные, которые лежат выше верхнего уса на нашем графике, то есть мы пока не будем рассматривать квартиры с очень высокой стоимостью на графике. Напомним, что «усы» ящика простираются влево и вправо от границ ящика на расстояние, равное 1,5 межквартильного размаха. В размах усов попадают нормальные значения, а за пределами находятся выбросы. 

In [ ]:
#третий квартиль + полтора межквартильного размаха
dfq = df.query('last_price <= @q3 + 1.5 * @iqr') 

In [ ]:
#Посмотрим, сколько данных мы отрезали:
f'Мы отрезали {1 - dfq.shape[0] / df.shape[0]:.1%} данных.'

Это довольно много. Элитное жилье можно рассмотреть отдельно.

Сделаем срез данных - создадим отдельную переменную для сегмента элитного жилья.

In [ ]:
high_price = df.query('last_price >= @q3 + 1.5 * @iqr')  

Теперь снова построим гистограмму для стоимости квартир без учета выбросов - данных, которые у нас лежали выше верхнего уса.

In [ ]:
plt.figure(figsize=(10,4))

sns.histplot(data=dfq, x='last_price', kde=True)
plt.title('\n Распределение стоимости квартир без учета выбросов\n')
plt.ylabel('Количество квартир') #чтобы убрать автоматические метки для осей
plt.xlabel('\nЦена квартиры, *10 в 7 степени, млн.рублей')
plt.show()

<div style="border:solid green 2px; padding: 20px"> 

Вывод:
    
После удаления выбросов график выглядит как нормальное распределение. 

Большинство квартир в датасете имеют стоимость в районе 3 млн - 6 млн. 

У 50% квартир в выборке цена меньше, чем 4 650 000 млн. 

Не более 1% квартир имеют стоимость выше 36 000 000 млн. рублей.

### Общая площадь квартиры

Посмотрим на описательную статистику значений столбца `total_area_m2`.

In [ ]:
df['total_area_m2'].describe()

Половина объектов имеют общую площадь не больше 52 м2. На среднее значение общей площади, которое составляет 60 м2, влияют выбросы. Например, есть объект с общей площадью в 900 м2.

Построим гистрограмму для площади квартир c помощью метода **hist()** библиотеки matplotlib. 

Уберем показ редких значений и посмотрим ближе распределение значений в диапазоне от 20 до 180.

In [ ]:
df.hist('total_area_m2', bins=100, range=(20,180), figsize=(10, 4), ec='black')
plt.title('Распределение квартир по общей площади \n')
plt.xlabel('\n Общая площадь квартиры')
plt.ylabel('Количество квартир')
plt.show();

<div style="border:solid green 2px; padding: 20px"> 

Вывод:

Мы убрали показ выбросов и хорошо видно, что чаще всего в датасете встречаются квартиры с площадью в диапазоне от 30 до 47 м2.

### Количество комнат

In [ ]:
df['number_of_rooms'].describe()

Построим гистограмму распределения количества комнат.

In [ ]:
df['number_of_rooms'].hist(figsize=(10, 4),ec='black')
plt.xticks(np.arange(0, 19, step=1)) 
plt.title('Распределение квартир по количеству комнат\n')
plt.xlabel('\nКоличество комнат')
plt.ylabel('Общее количество квартир\n')
plt.show()

Поизучаем квартиры с количеством комнат 0.

In [ ]:
print('Количество квартир с нулевым числом комнат: ', df.query('number_of_rooms == 0').shape[0])

Видимо, это квартиры-студии в большинстве случаев:

Посмотрим, сколько из них - квартиры-студии:

In [ ]:
df.query('number_of_rooms == 0 and is_studio == True').shape[0]

In [ ]:
f"Процент квартир с 0 количеством комнат: {df.query('number_of_rooms == 0').shape[0]/df.shape[0]:.2%}"

In [ ]:
f"Процент квартир с  количеством комнат 2: {df.query('number_of_rooms <=2').shape[0]/df.shape[0]:.2%}"

<div style="border:solid green 2px; padding: 20px"> 

Вывод:
    
В нашем датасете больше всего одно-, двух- и трехкомнатных квартир. 

50 % квартир имеет не больше 2 комнат.  
    
0.8% квартир имеют нулевое количество комнат - в большинстве случаев это квартиры-студии.

###  Высота потолков

In [ ]:
df['ceiling_height_m'].describe()

In [ ]:
df['ceiling_height_m'].hist(range=(2,3), figsize=(10, 4), ec='black')
plt.title('Распределение квартир по высоте потолков\n')
plt.xlabel('\nВысота потолков')
plt.ylabel('Общее количество квартир\n')
plt.show()

<div style="border:solid green 2px; padding: 20px"> 

Вывод:
    
На гистограмме видно, что больше всего квартир у нас имеют высоту потолков в районе 2.5 - 2.7 метров.

### Длительность публикации объявления

Изучим значения в days_exposition

In [ ]:
df['days_exposition'].describe()

Медиана (95) и среднее (180.9) существенно различаются.

Построим гистаграмму распределения длительности публикации объявления.

In [ ]:
df['days_exposition'].hist(bins=70, figsize=(10, 4), ec='black')
plt.xticks(np.arange(0, 1600, step=100)) 
plt.title('Распределение квартир по длительности публикации объявления\n')
plt.xlabel('\nДлительность публикации объявления о продаже')
plt.ylabel('Общее количество квартир\n')
plt.show()

Имеем дело со скошенным распределением - все сгруппировано слева и есть длинный хвост из выбросов справа. Есть пик на отметке в 45-50 дней. Возможно, объявления спустя 50 дней снимаются автоматически сервисом Я.Недвижимости.

На гистрограмме мы видим, что чаще всего квартиры продаются за примерно 100 дней (3 месяца). Крайне редко продажа длится более 450 дней, более года. Это необычно долгие продажи.  

<div style="border:solid green 2px; padding: 20px"> 

Вывод:

Примерно в 50% случаев квартиры продаются не дольше, чем за 100 дней, то есть за примерно 3 месяца с небольшим. 

Четверть всех объявлений имеет длительность продажи больше 232 дней. 
    
Крайне редко объявления о продаже имеют длинную длительность от 460 дней до 1580 дней, то есть больше года.  

## Определяем редкие и выбивающиеся значения

Для определения аномалий и редкий значений в данных попробуем использовать в том числе теорию ящика с усами.

«Усы» простираются влево и вправо от границ ящика на расстояние, равное 1,5 межквартильного размаха (сокращённо IQR, от англ. interquartile range, «межквартильный размах»). В размах «усов» попадают нормальные значения, а за пределами находятся выбросы, изображённые точками.

Попробуем создать функцию, которая будет определять для каждого задаваемого нами признака - границы "усов". Эти границы и будут выбросами для данного признака.

In [ ]:
"""
Функция на вход принимает имя датафрейма и имя столбца - изучаемый нами признак.
Возвращает границы левого и правого усов (список) на основе следующей логики:
Левый (или нижний) ус - это 4 квартиль минут полтора межквартильного разамаха,
а правый (или верхний ус) - это 3 квартиль плюс полтора межкартильного размаха.

"""
def get_whiskers(dataframe, column):
    q1 = df[column].quantile(0.25) # первый квантиль
    q3 = df[column].quantile(0.75) # третий квантиль
    minimum = df[column].min() # минимальное значение в наборе данных
    maximum = df[column].max() # максимальное значение в наборе данных

    # считаем межквартильный размах
    iqr = q3 - q1
    
    # считаем левый и правый ус, округляем значения    
    left_whisker = round(q1 - 1.5 * iqr, 2)
    right_whisker = round(q3 + 1.5 * iqr, 2)

    # левый ус не может быть меньше минимума:
    if left_whisker < minimum: left_whisker = minimum 
        
    # правый ус не может быть больше максимума:
    if right_whisker > maximum: right_whisker = maximum
        
    return [left_whisker, right_whisker]



### Общая площадь квартиры

In [ ]:
df['total_area_m2'].describe()

Построим диаграмму размаха для площади квартиры

In [ ]:
sns.boxplot(x=df['total_area_m2'])
plt.title('Диаграмма размаха общей площади')
plt.xlabel('\nОбщая площадь, кв.м')
plt.show()

In [ ]:
#Протестируем нашу функцию:
get_whiskers(df, 'total_area_m2')

У нас левый ус - это и есть наш минимум. Похоже, это квартира-студия. Квартиры с такой площадью существуют, поэтому мы не считаем это аномальными данными.
114.5 - это тоже не аномально большая площадь, если речь идет об элитном жилье или квартире с большим количеством комнат. Примерно после 350 плотность выбросов становится меньше. Возьмем отметку в 350 кв.м.

Создадим столбец со значениями True и False. Квартиры с очень большой площадью будут иметь True.

In [ ]:
#Все квартиры, где площадь больше или равна 350, будем считать очень большими по площади.
df['is_too_large_area'] = df['total_area_m2']>= 350

Проверяем, что у нас получилось.

In [ ]:
df['is_too_large_area'].value_counts()

In [ ]:
len(df[df['total_area_m2']>= 350])

In [ ]:
df[df['total_area_m2']==900][['total_area_m2', 'is_too_large_area']]

In [ ]:
f"Выбивающиеся и редкие значения от всей выборки составляют {df.query('total_area_m2 >= 350').shape[0] / df.shape[0]:.1%} данных"

### Cтоимость квартиры

In [ ]:
df['last_price'].describe()

Посмтрим на границы усов для признака цена.

In [ ]:
get_whiskers(df, 'last_price')

Квартира стоимостью 12190 смущает. Посмотрим на нее поближе.

In [ ]:
df[df['last_price'] == 12190]

Посмотрим на всю строку целиком.

In [ ]:
df.loc[8793]

Это очень странная квартира: слишком маленькая цена, слишком маленькая жилая площадь при наличие двух комнат и подозрительное соотношение жилой площади к общей.  

Посмотрим еще гистограмму, определив интервал по нашим границам усов.

In [ ]:
df['last_price'].hist(bins=100, range=(12190, 11897500));

Совсем мало значений на границе с 400 000. Проверим это c помощью условия в query

In [ ]:
df.query('last_price<=400000')

Найдем 99 персентиль. 

In [ ]:
df['last_price'].quantile(0.99)

Лишь 1% в нашей выборке квартир имеет стоимость выше 36 000 000 - возьмем эту отметку в качестве верхней границы.

Проверяем еще раз, сколько данных в итоге отрезаем:

In [ ]:
f"Выбивающиеся и редкие значения от всей выборки составляют {df.query('last_price >= 36000000 | last_price <= 400000').shape[0] / df.shape[0]:.1%} данных"


Все квартиры, где стоимость больше или равне 36 млн или меньше 400 тыс будем маркировать признаком с необычной ценой.

In [ ]:
df['is_unusual_price'] = (df['last_price']>= 36000000) | (df['last_price'] <= 400000)

Проверим, что получилось.

In [ ]:
df.loc[8793][['last_price', 'is_unusual_price']]

In [ ]:
df.loc[0][['last_price', 'is_unusual_price']]

In [ ]:
df.query('last_price >= 36000000 | last_price <= 400000').shape[0]

In [ ]:
df['is_unusual_price'].value_counts()

Редкими наблюдениями будем считать квартиры, где стоимость больше или равне 36 млн или меньше 400 тыс.

### Количество комнат

In [ ]:
df['number_of_rooms'].describe()

Cтроим ящик с усами для признака "количество комнат".

In [ ]:
sns.boxplot(x= df['number_of_rooms'])
plt.title('Диаграмма размаха количества комнат\n')
plt.xlabel('\nКоличество комнат')
plt.xticks(np.arange(0, 19, step=2)) 
plt.show();

На ящике видны выбросы за пределами верхнего (правого) уса. Проверим границы усов с помощью нашей функции get_whiskers.

In [ ]:
get_whiskers(df, 'number_of_rooms')

Запишем в новый столбец информацию о квартирах с очень большим количеством комнат.

In [ ]:
df['is_too_many_rooms'] = (df['number_of_rooms'] >= 6)

Редкими наблюдениями в столбце с количеством комнат будем считать объекты недвижимости с количеством комнат больше 6.

In [ ]:
f"Выбивающиеся и редкие значения от всей выборки составляют {df.query('number_of_rooms >= 6').shape[0] / df.shape[0]:.1%} данных"

Проверим, что получилось.

In [ ]:
df['is_too_many_rooms'].value_counts()

In [ ]:
df.query('number_of_rooms >= 6').shape[0]

### Высота потолков

In [ ]:
df['ceiling_height_m'].describe()

In [ ]:
sns.boxplot(data = df, y = 'ceiling_height_m')
plt.ylim(1, 6)
plt.title('Диаграмма размаха\n')
plt.ylabel('Высота потолков')
plt.show();

Посмотрим на значения границ усов.

In [ ]:
get_whiskers(df, 'ceiling_height_m')

Выбросы после 3 все-таки довольно плотные. Для элитных домов высота потолков в 4 метра не является редким значением.  

In [ ]:
f"Выбивающиеся и редкие значения от всей выборки составляют {df.query('ceiling_height_m <= 2.25 | ceiling_height_m >= 4').shape[0] / df.shape[0]:.1%} данных"

Все квартиры, где высота потолков больше или равна 4 кв. м. или меньше 2.25 будем маркировать признаком unusual_ceiling.

In [ ]:
df['is_unusual_ceiling'] = (df['ceiling_height_m']>= 4) | (df['ceiling_height_m'] <= 2.25)

Проверяем, что у нас получилось.

In [ ]:
len(df[(df['ceiling_height_m']>= 4) | (df['ceiling_height_m'] <= 2.25)])

In [ ]:
df['is_unusual_ceiling'].value_counts()

In [ ]:
df.loc[22590][['ceiling_height_m', 'is_unusual_ceiling']]

### Время продажи

In [ ]:
df['days_exposition'].describe()

In [ ]:
#Cмотрим на границы усов
get_whiskers(df, 'days_exposition')

Посмотрим на ящик с усами.

In [ ]:
sns.boxplot(x = df['days_exposition'])
plt.title('\n Диаграмма размаха длительности размещения объявления\n')
plt.xlabel('Длительность размещения публикации в днях\n')
plt.show()

Посмотрим, сколько процентов от выборки составляют квартиры, проданные за 1 день.

In [ ]:
f"Процент квартир, проданных за 1 день: {len(df.query('days_exposition <= 9'))/len(df):.1%}"

Необычно короткой продажей будем считать продажи за 1 день, необычно длинные - продажи, которые шли более 512 дней.

In [ ]:
#Cохраним в отдельный столбец признак необычно долгой или короткой публикации:
df['is_unusual_days_expos'] = (df['days_exposition'] == 1) | (df['days_exposition'] >= 512)

Проверим, что получилось.

In [ ]:
df['is_unusual_days_expos'].value_counts()

In [ ]:
len(df[(df['days_exposition'] == 1) | (df['days_exposition'] >= 512)])

In [ ]:
df.loc[2][['days_exposition', 'is_unusual_days_expos']]

In [ ]:
f"Выбивающиеся и редкие значения от всей выборки составляют {df.query('days_exposition ==1 | days_exposition >= 512').shape[0] / df.shape[0]:.1%} данных"

### Вывод


Мы попытались определить редкие и выбивающиеся значения для следующих признаков: общая площадь объекта недвижимости, стоимость, число комнат, высота потолков.

* Аномально большой по общей площади объект недвижимости имеет площадь 900 м.кв. Объекты с площадью в 350 м.кв. и больше считаются редкими и составляют 0.17% от всей выборки.

* Аномально дорогой объект недвижимости имеет стоимость 763 000 000 млн. рублей. Аномально дешевый объект недвижимости имеет стоимость в 12 190 рублей.
 В целом объекты недвижимости дороже 36 000 000 млн рублей и дешевле 400 000 тыс рублей составляют 1 % от выборки. 

* Аномально редкое количество комнат - 19. В целом объекты с количеством комнат больше 6 встречаются редко, их процент составляет 0.8% от всей выборки. 

* Аномально низкий потолок - 1 метр. Аномально высокий - 100 метров. Объекты недвижимости с высотой потолка больше или равной 4 кв.м или меньше 2.25 считаются редкими и составляют 0.6 % от всей выборки.

* Аномально короткий срок длительности публикации о продаже недвижимости - 1 день. Аномально длинный - 1580 дней (4 года 4 месяца). 
Объекты недвижимости, срок длительности публикации у которых равен 1 дню или больше или равен 512 дням (1 год 5 месяцев), считаются редкими и выбивающимися. Такие случаи составляют 7.1% от всей выборки.




## Определяем, какие факторы больше всего влияют на стоимость квартиры. 

Изучим, зависит ли цена от площади, количества комнат, удалённости от центра. 

Изучим зависимость цены от того, на каком этаже расположена квартира: первом, последнем или другом, а также влияют ли даты размещения: день недели, месяц и год.

Для начала мы уберем редкие и выбивающиеся значений - отфильтруем наши данные, потому что коэффициент Пирсона к выбросам очень чувствительный:

Фильтруем данные без выбросов в отдельный датафрейм.

In [ ]:
clean_df = df[(df['is_too_large_area'] == False) & 
     (df['is_unusual_price'] == False) &
     (df['is_too_many_rooms'] == False) &
     (df['is_unusual_ceiling'] == False) &
     (df['is_unusual_days_expos'] == False)].reset_index(drop=True)
    

f'Доля чистых значений составила: {len(clean_df) / len(df):.1%}'

То есть мы отрезали примерно 9 % данных. Это допустимо. 

Посмотрим на коэффициент корреляции Пирсона для таких признаков, как общая площадь квартиры, жилая площадь квартиры, количество комнат, удаленность от центра, высота потолков.

In [ ]:
#Построим матрицу корреляции:
cols = ['total_area_m2', 'living_area_m2', 'ceiling_height_m', 'number_of_rooms', 'last_price', 'city_center_distance_m']
(clean_df[cols].corr()
.style
.format("{:.1f}")
.background_gradient(cmap='Blues'))

Близкий к 1 коэффициент только у пары 'стоимость-общая площадь' (тут сильная корреляция, коэффициент Пиросна равен 0.8), чуть слабее корреляция у пары 'стоимость - жилая площадь (0.6)". Корреляция в 0.5 у пары 'стоимость-количество комнат' говорит о наличии связи, однако слабой. То же самое можно сказать и о паре "стоимость - высота потолков".

Нарисуем диаграмму рассеяния для цены и общей площади, чтобы визуализировать взаимосвязь.

In [ ]:
clean_df.plot(x='last_price',y='total_area_m2',kind='scatter') 
plt.title('Зависимость стоимости квартиры от общей площади \n')
plt.xlabel('\n Стоимость квартиры')
plt.ylabel('Общее количество квартир\n')
plt.show();

<div style="border:solid green 2px; padding: 20px">  
    
#### Взаимосвязь между стоимостью и общей площадью. Промежуточный вывод: 

Коэффициент Пирсона равен 0.8. Это высокая корреляция. Видно, что линейная положительная связь есть, то есть **чем больше площадь, тем дороже объект недвижимости**.

Для значений столбцов с днями недели, месяцами, годами и категорией этажа нет смысла смотреть на коэффициент корреляции Пирсона. Это дискретные величины и к ним он не применим. 

Создадим функцию для построения сводной таблицы по интересующему нас признаку и для отрисовки графика.

In [ ]:
"""
Данная функция берет на вход два строковых значения: признак (feature), то есть имя столбца в датасете, по которому мы будем группировать данные, и 
название этого признака на русском языке (feature_name) для корректных подписей на графике. 
В теле функции мы строим сводную таблицу по данному признаку, вычисляем медиану стоимость квартиры в каждой группе и выводим сводную таблицу на экран,
а также по этой сводной таблице мы строим график и выводим его на экран.

"""

def get_pivot_and_plot(feature, feature_name):
    pivot_table = clean_df.groupby(feature)['last_price'].agg(['median'])
    display(pivot_table.sort_values(by='median').style.format("{:.1f}").background_gradient(cmap='Blues'))
    pivot_table.plot()
    plt.title(f'\n Зависимость стоимости квартиры от признака "{feature_name}"\n')
    plt.ylabel('Стоимость квартиры')
    plt.xlabel(f'\n{feature_name}')
    plt.show();

Посмотрим, как меняется стоимость квартиры в зависимости от категории этажа.

In [ ]:
get_pivot_and_plot('floor_category', 'категория этажа')

<div style="border:solid green 2px; padding: 20px">  

#### Взаимосвязь между стоимостью и этажом. Промежуточный вывод: 

Самая низкая стоимость у объектов, которые находятся на первом этаже. 

Самая высокая - у объектов, которые находятся не на первом и не на последнем этаже.

Посмотрим, как зависимость между публикацией объявления о продаже квартиры и стоимостью квартиры.

Напомним, что 0 - это понедельник. 

In [ ]:
get_pivot_and_plot('weekday', 'день недели')

<div style="border:solid green 2px; padding: 20px"> 

#### Взаимосвязь между днем недели публикации объявления. Промежуточный вывод: 
Самая высокая стоимость у квартир, объявления о продаже которых опубликовано в понедельник, вторник и среду. 

Стоимость квартир, опубликованных в четверг, резко падает, и квартиры, опубликованные в пятницу, субботу и воскресенье имеют примерно одинаковую стоимость. 

Посмотрим, как меняется стоимость квартиры в зависимости от месяца публикации.

In [ ]:
get_pivot_and_plot('month', 'месяц публикации объявления о продаже квартиры')

<div style="border:solid green 2px; padding: 20px"> 

#### Взаимосвязь между месяцем публикации объявления. Промежуточный вывод: 

У объектов, объявление о продаже которых опубликовано c января по апрель, стоимость растет и достигает пика в апреле. В мае отмечается резкое падение цены, которое достигает дна в июне. Дальше происходит постепенный рост со спадами. 

Посмотрим, как меняется стоимость квартиры в зависимости от года публикации.

In [ ]:
get_pivot_and_plot('year', 'год публикации объявления о продаже квартиры')

<div style="border:solid green 2px; padding: 20px"> 

#### Взаимосвязь между годом публикации объявления. Промежуточный вывод: 
В 2014 году цена была самой высокой и падала до 2016 года. С 2016 по 2018 была примерно на одном и том же уровне и с 2018 немного подроcла, почти достигнув уровня 2015 года.

## Десять населённых пунктов с наибольшим количеством объявлений.

Сгруппируем наши данные по названию населенного пункта, посчитаем среднюю цену квадратного метра, покажем топ-10 населенных пунктов по количеству объявлений.

In [ ]:
locality_groupped = df.groupby('locality_name')['price_per_m2'].agg(['count', 'mean']).sort_values(by=['count'], ascending=False)
locality_groupped.head(10).style.format("{:.1f}").background_gradient(cmap='Blues')

Построим столбчатую диаграмму, которая покажет десятку городов, где самая высокая цена за 1 кв.м.

In [ ]:
plt.figure(figsize=(10,4))
sns.set_palette('pastel')
sns.set_theme(style="darkgrid")
sns.barplot(x='mean', y='locality_name', orient='h', data=locality_groupped.head(10).sort_values(by='mean', ascending=False).drop(columns=['count']).reset_index())
plt.title('Распределение квартир по цене за 1 кв м \n',fontsize=20)
plt.ylabel('')
plt.yticks(fontsize=14)
plt.xlabel('\nСредняя цена за 1 кв м, руб\n')
plt.xticks(fontsize=14)
plt.show();

Посмотрим, сколько всего населенных пунктов в нашем датафрейме (помним, что есть и unknown).

In [ ]:
f'Всего населенных пунктов в датафрейме: {locality_groupped.shape[0]}.'

### Вывод



* Всего в нашем датафрейме не меньше 331 населенных пунктов.
    
* Лидер с наибольшим числом объявлений - город Санкт-Петербург. 
    
* Самое дорогое жилье в 10-ке лидеров по количеству объявлений - в Санкт-Петербурге (средняя стоимость за 1 кв. м составляет 114 868 руб.), а самое дешевое - в Выборге (средняя стоимость за 1 кв.м. составляет - 58 141 рублей).

<a name="stage_5"></a>

# Этап 5. Исследование данных о квартирах Санкт-Петербурга


[Наверх к оглавлению](#contents)

Выделим квартиры в Санкт-Петербурге. 
Наша первая задача — выяснить, какая область входит в центр. 

Столбец city_center_distance_m — это расстояние до центра города в метрах. Для удобства создадим столбец с расстоянием до центра в километрах, окргулим до целых значений. 


In [ ]:
df.columns

In [ ]:
df['city_center_distance_km'] = round(df['city_center_distance_m'] / 1000, 0)
df['city_center_distance_km']

Теперь сделаем срез данных для квартир в Санкт-Петербурге: уберем редкие, выбивающиеся значения, строки с пустым значением в city_center_distance_km.

In [ ]:
#Фильтруем данные без выбросов для Санкт-Петербурга в отдельный датафрейм:
clean_df_spb = df[(df['is_too_large_area'] == False) & 
     (df['is_unusual_price'] == False) &
     (df['is_too_many_rooms'] == False) &
     (df['is_unusual_ceiling'] == False) &
     (df['is_unusual_days_expos'] == False) &
     (df['locality_name'] == 'Санкт-Петербург') &
     (df['city_center_distance_km'].notnull())].reset_index(drop=True)
    
print('Доля чистых значений составила: {:.1%}'. format(len(clean_df_spb) / len(df)))

Проверим, верно ли отфильтровались квартиры по населенному пункту.

In [ ]:
clean_df_spb['locality_name'].value_counts()

Все в порядке.

Сгруппируем данные по расстоянию до центра, посчитаем среднюю цену для каждого километра

In [ ]:
df.columns

In [ ]:
spb_center_nearest = clean_df_spb.groupby('city_center_distance_km')['price_per_m2'].agg('mean')
spb_center_nearest

Построим график: он должен показывать, как цена зависит от удалённости от центра. Определим границу, где график сильно меняется, — это и будет центральная зона.

In [ ]:
#Cтроим простенький плот - его нам достаточно, чтобы увидеть границу.
spb_center_nearest.plot();

Пик цены около 1 км, дальше цена резко падает, после 4 км снова растет и после 8 километра устойчивая тенденция к понижению. Будем считать 8 километр - границей центральной зоны.
На 27 есть всплеск роста (наверное, это некоторые выбивающиеся данные или, возможно, что-то еще очень сильно повлияло на рост цены). 

1. Теперь выделим сегмент квартир в центре Санкт-Петербурга. 

2. Проанализируем эту территорию и изучим следующие параметры: площадь, цена, число комнат, высота потолков. 

3. Определим факторы, которые влияют на стоимость квартиры (число комнат, этаж, удалённость от центра, дата размещения объявления).

4. Сделаем вывод отличаются ли они от общих выводов по городу.

Выделяем сегмент квартир в центре Санкт-Петербурга.

In [ ]:
spb_center = clean_df_spb[clean_df_spb['city_center_distance_km'] <= 8]
spb_center.head()

### Анализ и сравнение площади, цены, количества комнат, высоты потолков в центре Санкт-Петербурга и для остальных районов города.

Выведем описательную статистику для данных по центру Санкт-Петербурга и всему городу.

In [ ]:
cols = (['total_area_m2', 'last_price', 'number_of_rooms', 'ceiling_height_m'])
for col_name in cols:
    print(f'{col_name} для центра Санкт-Петербурга:', '\n')
    print(spb_center[col_name].describe(), '\n')
    print(f'{col_name} для всего Санкт-Петербурга:', '\n')
    print(clean_df_spb[col_name].describe(), '\n')
    

Построим столбчатые диаграммы по каждому признаку с помощью библиотеки seaborn. Для наглядности сравнения построим данные для центра Санкт-Петербурга и остальных районов города на одном графике. Нормируем показатели через параметр stat="density".     

In [ ]:
plt.figure(figsize=(15, 10))

for i, col in enumerate(['total_area_m2', 'last_price', 'number_of_rooms', 'ceiling_height_m']):

    plt.subplot(2, 2, i+1)
    sns.histplot(
        clean_df_spb, x=col, element="step",
        stat="density", 
        ).set(title=col)
    sns.histplot(
        spb_center, x=col, element="step",
        stat="density",
        color='orange'
        )
    
plt.legend(['Весь Санкт-Петерубург','Центр Санкт-Петербурга'], loc = 'upper right', bbox_to_anchor=(0.1, -.2))
plt.suptitle('Сравнение параметров квартир из всего Санкт-Петербурга и его центра')
plt.show()

#### Вывод 

 

* Цена и общая площадь у квартир в центре Санкт-Петербурга в среднем выше, чем по всему городу. 

* В центре гораздо больше квартир с большим числом комнат (от 4 и до 6). Для всего города этот процент существенно ниже.
    
То же самое можно сказать и о двушках и трешках. Их почти равное количество в центре города. В то время как во всем городе больше половины квартир - это двушки. 

* Высота потолков в квартирах в центре в среднем выше, чем по всему городу.

### Анализ факторов, влияющих на стоимость квартиры в центре Санкт-Петербурга и в остальных районах города.

Выделим факторы, которые влияют на стоимость квартиры в центре города (число комнат, этаж, удалённость от центра, дата размещения объявления, общая площадь). 

In [ ]:
#Построим матрицу корреляции для центра Санкт-Петербурга:
cols = ['last_price', 'total_area_m2', 'number_of_rooms', 'city_center_distance_m', 'ceiling_height_m', 'living_area_m2','floor']
print ('\n Матрица корреляции для центра Санкт-Петербурга: \n')
(spb_center[cols].corr()
.style
.format("{:.1f}")
.background_gradient(cmap='Blues'))

In [ ]:
spb_center[cols].corr()['last_price']

Посмотрим матрицу для всего города.

In [ ]:
print ('\n Матрица корреляции для всего Санкт-Петербурга:\n')
(clean_df_spb[cols].corr()
.style
.format("{:.1f}")
.background_gradient(cmap='Blues'))

**Промежуточный вывод:** 

* есть высокая положительная корреляция (0.7, 0.8) между площадью и ценой для центра, для всего города

* есть положительная корреляция между числом комнат (0.4, 0.5) и ценой (еще проверим на графике) для центра, для всего города.

* есть отрицательная слабая корреляция (-0.1, -0.4) между расстоянием до центра и стоимостью квартиры для всего города.

Отдельно посмотрим и на категорию этажа, влияет ли она на стоимость квартир в центре Санкт-Петербурга:

In [ ]:
spb_center.groupby('floor_category')['last_price'].mean().plot();

**Промежуточный вывод**:

    Да, зависимость есть. Самые дешевые квартиры - на первом этаже. Самые дорогие - не на первом и не на последнем.

In [ ]:
#Линейная диаграмма для центра - дата размещения публикации по оси абцисс, стоимость - по оси ординат.
spb_center.groupby('first_day_exposition')['last_price'].mean().plot();

Диаграмма неинформативна. Тут никакой зависимости не видно.

Посмотрим отдельно на день недели, месяц, год - есть ли какая-то зависимость от этих параметров. 

In [ ]:
spb_center.groupby('year')['last_price'].mean().plot();

In [ ]:
spb_center.groupby('weekday')['last_price'].mean().plot();

In [ ]:
spb_center.groupby('month')['last_price'].mean().plot();

**Промежуточный вывод**: 

* Самая высокая цена на квартиры в Санкт-Петербурге была в 2014 году, цена падала до 2015 года, с 2015-2016 цена особо не менялась, с 2016 снова пошла на спад, с 2017 по 2018 держалась на одном (самом низком уровне) и с 2018 снова начала немного расти.

* Самая низкая цена у квартир, объявление о продаже которых опубликовали в пятницу.

* Квартиры, объявления о продаже которых опубликовали в ноябре, резко падают в цене и в декабре цена достигает дна.

Линейные диаграммы для проверки, влияет ли число комнат на стоимость.

In [ ]:
spb_center.groupby('number_of_rooms')['last_price'].mean().plot(
    title='Зависимость между числом комнат и ценой для центра Санкт-Петербурга \n', 
    ylabel='Стоимость', xlabel='Число комнат' );

In [ ]:
clean_df_spb.groupby('number_of_rooms')['last_price'].mean().plot(
    title='Зависимость между числом комнат и ценой для всего Санкт-Петербурга \n', 
    ylabel='Стоимость', xlabel='Число комнат' );

#### Вывод

* И для центра, и для всего города есть положительная корреляция между площадью квартиры и ее стоимостью. Коэффициенты Пирсона равны 0,7 и 0,8 соответственно. Чем больше площадь, тем  дороже квартиры.

* Наблюдается зависимость между категорией этажа и стоимостью как для центра, так и для всего города: квартиры на первом этаже самые дешевые. Самые дорогие - это те квартиры, которые находятся не на первом и не на последнем этажах.

* Количество комнат влияет на стоимость квартиры: чем больше комнат - тем выше стоимость. 

* Для всего Санкт-Петербурга есть зависимость (более слабая) между расстоянием до центра и стоимостью квартиры. Чем дальше от центра, тем квартиры дешевле. 

* Самая высокая цена на квартиры в Санкт-Петербурге была в 2014 году. Самые низкие цены были в 2017-2018 гг.

<a name="stage_6"></a>

# Этап 6. Общий вывод

[Наверх к оглавлению](#contents)

**Чтобы определить рыночную стоимость объектов недвижимости следует учитывать следующие параметры:**

* Общая площадь. Объекты с общей площадью в 350 м.кв. и больше считаются редкими. Аномально редкое значение - 900 м.кв. В среднем площадь составляет от 30 до 47 м2 кв.м. Чем больше площадь, тем выше стоимость.

* Количество комнат. Объекты с количеством комнат больше 6 встречаются редко. Аномально редкое значение - 19. Чем больше комнат, тем выше стоимость. 

* Этаж. Самые дешевые объекты на первом этаже. Самые дорогие - на других этажах, кроме первого и последнего.

* Расстояние до центра. Центр города Санкт-Петербурга ограничен окружностью радиусом в 8 км. Для всего города фактор близости слабо, но влияет на стоимость жилья: чем дальше от центра, тем дороже объект недвижимости.

* Высота потолка. Объекты недвижимости с высотой потолка больше или равной 4 кв.м или меньше 2.25 считаются редкими. В среднем высота потолков: 2.5 - 2.7 метров. Этот фактор на стоимость жилья влияет довольно слабо.

* Населенный пункт. Самый дорогой объект недвижимости  - в крупном городе, в Санкт-Петербурге.


**При построении автоматизированной системы, которая детектирует аномалии и мошенническую деятельность, следует учесть следующие аномальные значения параметров:**

* Аномально большой по общей площади объект недвижимости имеет площадь 900 м.кв. Объекты с площадью *в 350 м.кв. и больше* считаются редкими и составляют 0.17% от всей выборки в данном исследовании.

* Аномально дорогой объект недвижимости имеет стоимость 763 000 000 млн. рублей. Аномально дешевый объект недвижимости имеет стоимость в 12 190 рублей. В целом объекты недвижимости дороже 36 000 000 млн рублей и дешевле 400 000 тыс рублей составляют 1 % от выборки.

* Аномально редкое количество комнат - 19. В целом объекты с количеством комнат *больше 6* встречаются редко, их процент составляет 0.8% от всей выборки в данном исследовании.

* Аномально низкий потолок - 1 метр. Аномально высокий - 100 метров. Объекты недвижимости с высотой потолка *больше или равной 4 кв.м или меньше 2.25* считаются редкими и составляют 0.6 % от всей выборки в данном исследовании.

* Аномально короткий срок длительности публикации о продаже недвижимости - 1 день. Аномально длинный - 1580 дней (4 года 4 месяца). Объекты недвижимости, срок длительности публикации у которых равен 1 дню или больше или равен 512 дням (1 год 5 месяцев), считаются редкими и выбивающимися. Такие случаи составляют 7.1% от всей выборки.

# Спасибо за внимание!